In [5]:
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import SH
import pandas as pd

# === CONFIGURATION ===
input_file = "config.ttl"  # Remplace par ton chemin d'entrée
output_file = "config_nonsignificant.ttl"

# Namespace de base
BASE = Namespace("https://data.mydomain.com/ontologies/sparnatural-config/")
prefix = BASE

# Charger le graphe
g = Graph()
g.parse(input_file, format="turtle")

# Stocker les correspondances anciennes → nouvelles URIs
replacements = []

# Chercher les sh:property avec URIRef
property_usages = list(g.triples((None, SH.property, None)))

counter = 1
for s, p, o in property_usages:
    if isinstance(o, URIRef) and str(o).startswith(str(prefix)):
        new_uri = URIRef(f"{prefix}p{counter}")
        counter += 1

        # Copier tous les triplets liés à l'ancienne URI
        for po, oo in g.predicate_objects(subject=o):
            g.add((new_uri, po, oo))

        # Remplacer dans le NodeShape
        g.remove((s, p, o))
        g.add((s, p, new_uri))

        # Supprimer les anciens triplets
        for po, oo in g.predicate_objects(subject=o):
            g.remove((o, po, oo))

        # Enregistrer le changement
        replacements.append({
            "Ancienne URI": str(o),
            "Nouvelle URI": str(new_uri)
        })

# Sauvegarder le nouveau TTL
g.serialize(destination=output_file, format="turtle")
print(f"✅ Fichier sauvegardé dans '{output_file}'")

# Afficher les correspondances
df = pd.DataFrame(replacements)
print("\n=== Tableau des correspondances ===\n")
print(df.to_string(index=False))

✅ Fichier sauvegardé dans 'config_nonsignificant.ttl'

=== Tableau des correspondances ===

                                                                     Ancienne URI                                                Nouvelle URI
            https://data.mydomain.com/ontologies/sparnatural-config/Artwork_label  https://data.mydomain.com/ontologies/sparnatural-config/p1
           https://data.mydomain.com/ontologies/sparnatural-config/Artwork_author  https://data.mydomain.com/ontologies/sparnatural-config/p2
     https://data.mydomain.com/ontologies/sparnatural-config/Artwork_creationYear  https://data.mydomain.com/ontologies/sparnatural-config/p3
      https://data.mydomain.com/ontologies/sparnatural-config/Artwork_displayedAt  https://data.mydomain.com/ontologies/sparnatural-config/p4
        https://data.mydomain.com/ontologies/sparnatural-config/Artwork_thumbnail  https://data.mydomain.com/ontologies/sparnatural-config/p5
      https://data.mydomain.com/ontologies/sparnatural-c